In [1]:
import glob, cv2
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
tf.config.list_physical_devices('GPU')

2025-09-03 14:37:44.668118: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-03 14:37:44.678832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-03 14:37:44.682715: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
dogs = glob.glob("cat_and_dog/Dog/*")
cats = glob.glob("cat_and_dog/Cat/*")

In [3]:
df = pd.concat([
    pd.DataFrame({"label":1,"path":dogs}),
    pd.DataFrame({"label":0,"path":cats})
], ignore_index = True).sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
def load_image(path, img_size):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    return img

In [5]:
class MyGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, function, batch_size = 32,
                 image_size = (224, 224), shuffle = True):
        
        self.df = df
        self.batch_size = batch_size
        self.img_size = image_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()
        self.function = function
    # 배치를 몇번 반복할지 반환
    def __len__(self):
        return int(np.ceil(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[
            index*self.batch_size : (index+1)*self.batch_size]
        batch_df = self.df.iloc[batch_indices]
        
        batch_images = np.array([self.function(
            p, self.img_size) for p in batch_df['path']])
        batch_labels = np.array(batch_df['label'])
        
        return batch_images, batch_labels
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

In [6]:
train_df, val_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label'])
test_df, val_df = train_test_split(
    val_df, test_size=0.5, random_state=42, stratify=val_df['label'])
train_gen = MyGenerator(train_df,load_image, batch_size=32)
val_gen = MyGenerator(val_df, load_image, batch_size=32)

In [7]:
backbone = tf.keras.applications.MobileNetV2(
    input_shape = (224,224,3), include_top = False)
backbone.trainable = False

inputs = tf.keras.layers.Input(shape=(224,224,3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = backbone(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

2025-09-03 14:37:44.862012: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-03 14:37:44.863164: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-03 14:37:44.867094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-09-03 14:37:44.869842: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [8]:
model = tf.keras.Model(inputs, outputs)

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [tf.keras.metrics.BinaryAccuracy()]
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    workers=4,                 # CPU 코어 수에 맞춰 조정
    use_multiprocessing=True,  # 병렬 처리 활성화
)

Epoch 1/10


2025-09-03 14:37:52.310265: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


  3/586 [..............................] - ETA: 48s - loss: 0.4379 - binary_accuracy: 0.7500    

2025-09-03 14:37:55.664313: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


231/586 [==========>...................] - ETA: 46s - loss: 0.0631 - binary_accuracy: 0.9785 

...

536/586 [==========================>...] - ETA: 6s - loss: 0.0013 - binary_accuracy: 0.9996 

Corrupt JPEG data: 254 extraneous bytes before marker 0xd9


586/586 [==============================] - ETA: 0s - loss: 0.0013 - binary_accuracy: 0.9995

Corrupt JPEG data: 399 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2230 extraneous bytes before marker 0xd9


586/586 [==============================] - 94s 158ms/step - loss: 0.0013 - binary_accuracy: 0.9995 - val_loss: 0.0520 - val_binary_accuracy: 0.9885


In [12]:
model.save("cat_and_dogs")

2025-09-03 14:55:57.365775: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: cat_and_dogs/assets


In [13]:
model.save("cat_and_dogs.h5", save_format="h5")

In [15]:
model.save("cat_and_dogs.keras")